First, we'll import pytorch and check if a GPU is available.

In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
from FrEIA.modules import *
from FrEIA.framework import *
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

if torch.cuda.is_available():
    print('GPU available')
else:
    print('CPU only')

CPU only


Next, define the path to the data we're using

In [2]:
data_path = Path('C:\\Users\\dohert01\\PycharmProjects\\qPAI_cINN_uncertainty_estimation\\datasets')
experiment_name = "FlowPhantom_insilico_complicated"

Let's have a look at the data. First of all, borrow some normalisation functions...

In [3]:
def spectrum_normalisation(spectrum):
    """Applies z-score scaling to the initial pressure spectrum"""
    mean = np.mean(spectrum)
    std = np.std(spectrum)
    norm = (spectrum - mean)/std
    return norm

def spectrum_processing(spectrum, allowed_datapoints):
    """Returns a normalised initial pressure spectrum with some of the values zeroed out"""
    num_non_zero_datapoints = random.choice(allowed_datapoints)
    a = np.zeros(len(spectrum))
    a[:num_non_zero_datapoints] = 1
    np.random.shuffle(a)

    incomplete_spectrum = list(np.multiply(a, np.array(spectrum)))
    non_zero_indices = np.nonzero(incomplete_spectrum)
    non_zero_values = list(filter(None,incomplete_spectrum))
    normalised_non_zero = spectrum_normalisation(non_zero_values)

    i = 0
    for index in non_zero_indices[0]:
        incomplete_spectrum[index] = normalised_non_zero[i]
        i+=1

    normalised_incomplete_spectrum = np.array(incomplete_spectrum)

    return normalised_incomplete_spectrum

def batch_spectrum_processing(batch, allowed_datapoints):
    processed = []

    for spectrum in batch:

        processed.append(spectrum_processing(spectrum, allowed_datapoints))
    return torch.tensor(np.array(processed))

Let's load the data from file

In [4]:
training_spectra_file = data_path / experiment_name / "training_spectra.pt"
validation_spectra_file = data_path / experiment_name / "validation_spectra.pt"
test_spectra_file = data_path / experiment_name / "test_spectra.pt"

training_oxygenations_file = data_path / experiment_name / "training_oxygenations.pt"
validation_oxygenations_file = data_path / experiment_name / "validation_oxygenations.pt"
test_oxygenations_file = data_path / experiment_name / "test_oxygenations.pt"

train_spectra_original = torch.load(training_spectra_file)
train_oxygenations_original = torch.load(training_oxygenations_file)
validation_spectra_original = torch.load(validation_spectra_file)
validation_oxygenations_original = torch.load(validation_oxygenations_file)
test_spectra_original = torch.load(test_spectra_file)
test_oxygenations_original = torch.load(test_oxygenations_file)

Now let's look at the dimensions

In [5]:
print(train_spectra_original.size())
print(train_oxygenations_original.size())
print(train_spectra_original[0])

torch.Size([134624, 41])
torch.Size([134624])
tensor([634.9278, 600.2585, 600.2339, 587.4062, 580.4452, 573.9892, 582.9027,
        597.7095, 601.8840, 641.6681, 655.6356, 704.5982, 730.0311, 739.1377,
        762.7631, 768.2003, 789.5642, 808.6349, 811.7870, 835.5294, 866.5328,
        886.8488, 918.7031, 905.1712, 913.7165, 913.7761, 913.4937, 919.7126,
        915.4688, 919.2101, 887.4873, 870.8792, 905.5049, 883.7628, 876.9416,
        888.4904, 881.3424, 888.5063, 892.4427, 879.3855, 869.1013],
       dtype=torch.float64)


In [6]:
# Zeroing out some of the spectrum data (randomly) and normalising
allowed_datapoints = [10]

train_spectra = batch_spectrum_processing(train_spectra_original, allowed_datapoints)
validation_spectra = batch_spectrum_processing(validation_spectra_original, allowed_datapoints)
test_spectra = batch_spectrum_processing(test_spectra_original, allowed_datapoints)

In [8]:
# Reshaping initial pressure spectra to fit LSTM input size
train_spectra = torch.reshape(train_spectra, (len(train_spectra), len(train_spectra[0]), 1))
validation_spectra = torch.reshape(validation_spectra, (len(validation_spectra), len(validation_spectra[0]), 1))
test_spectra = torch.reshape(test_spectra, (len(test_spectra), len(test_spectra[0]), 1))

train_oxygenations = torch.reshape(train_oxygenations_original,(len(train_oxygenations_original),1))
validation_oxygenations = torch.reshape(validation_oxygenations_original,(len(validation_oxygenations_original),1))
test_oxygenations = torch.tensor(np.float32(test_oxygenations_original))
test_oxygenations = torch.reshape(test_oxygenations_original,(len(test_oxygenations_original),1))

In [23]:
class MultiSpectralPressureO2Dataset(Dataset):
    def __init__(self, spectra, oxygenations, transform=None, target_transform=None):
        self.data = spectra
        self.labels = oxygenations
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = self.data[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [33]:
training_dataset = MultiSpectralPressureO2Dataset(train_spectra, train_oxygenations)
training_dataloader = DataLoader(training_dataset, batch_size=2048, shuffle=True)
data, label = next(iter(training_dataloader))
print(data[0])
print(label[0])

Time to define the model... (both LSTM and INN)

In [32]:
# Define the subnet for the invertible blocks (use the AllInOneBlack from FrEIA)
def subnet(dims_in, dims_out):
    return nn.Sequential(nn.Linear(dims_in, 256), nn.ReLU(),
                         nn.Linear(256, dims_out))
# Define the LSTM for the conditioning network
lstm_layer = nn.LSTM(41, 100, 1, batch_first=True)
# Define the GraphINN that combines the nodes
nodes = [InputNode(41, 1, name='input')]
conditions = [ConditionNode]
for i in range(n_blocks):
    nodes.append(AllInOneBlock(41, 1, subnet_constructor=))
GraphINN(nodes + conditions)

tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [-0.3367],
        [ 0.0000],
        [-0.4733],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 2.5467],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.6764],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.2532],
        [ 0.1733],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [-0.2104],
        [-0.7148],
        [-0.6127],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [ 0.0000],
        [-1.3017]], dtype=torch.float64)
tensor([0.4330], dtype=torch.float64)


Time to define the training loop...